# Visualizacion y calculo de proyecciones en 3D (coordenadas homogeneas)

Este notebook muestra:
- Representacion de puntos 3D en coordenadas homogeneas.
- Proyeccion ortogonal sobre el plano XY.
- Proyeccion en perspectiva con distancia focal `d`.
- Efecto de variar `d` en la proyeccion perspectiva.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

plt.style.use('seaborn-v0_8-whitegrid')
np.random.seed(7)


In [ ]:
# Vertices de un cubo desplazado para mantener z > 0
vertices = np.array([
    [-1, -1, 3],
    [ 1, -1, 3],
    [ 1,  1, 3],
    [-1,  1, 3],
    [-1, -1, 5],
    [ 1, -1, 5],
    [ 1,  1, 5],
    [-1,  1, 5],
], dtype=float).T  # shape: (3, 8)

# Nube adicional para ver mejor el efecto en varios puntos
n_random = 60
rand_x = np.random.uniform(-1.2, 1.2, n_random)
rand_y = np.random.uniform(-1.2, 1.2, n_random)
rand_z = np.random.uniform(2.5, 5.5, n_random)
nube = np.vstack([rand_x, rand_y, rand_z])

puntos_3d = np.hstack([vertices, nube])  # shape: (3, N)
vertices_idx = list(range(8))

# Aristas del cubo (indices sobre los 8 vertices)
aristas = [
    (0,1), (1,2), (2,3), (3,0),
    (4,5), (5,6), (6,7), (7,4),
    (0,4), (1,5), (2,6), (3,7)
]

fig = plt.figure(figsize=(7, 6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(puntos_3d[0], puntos_3d[1], puntos_3d[2], s=20, alpha=0.7, label='Puntos 3D')
ax.scatter(vertices[0], vertices[1], vertices[2], s=50, color='crimson', label='Vertices cubo')

for i, j in aristas:
    ax.plot([vertices[0, i], vertices[0, j]],
            [vertices[1, i], vertices[1, j]],
            [vertices[2, i], vertices[2, j]], color='crimson', linewidth=1.6)

ax.set_title('Escena original en 3D')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.legend(loc='upper left')
plt.show()


In [ ]:
def a_homogeneas(puntos):
    """Convierte puntos (3, N) a coordenadas homogeneas (4, N)."""
    return np.vstack([puntos, np.ones((1, puntos.shape[1]))])

def desde_homogeneas(puntos_h):
    """Normaliza por w y regresa coordenadas euclidianas."""
    return puntos_h[:-1] / puntos_h[-1]

def matriz_ortogonal_xy():
    """Proyeccion ortogonal sobre el plano XY."""
    return np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 1],
    ], dtype=float)

def matriz_perspectiva(d=1.0):
    """Proyeccion perspectiva con distancia focal d (d > 0)."""
    if d <= 0:
        raise ValueError('d debe ser mayor que 0')
    return np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 1/d, 0],
    ], dtype=float)

def proyectar(puntos, P):
    """Aplica una matriz 4x4 a puntos 3D (3, N) y regresa (3, N)."""
    puntos_h = a_homogeneas(puntos)
    proy_h = P @ puntos_h
    return desde_homogeneas(proy_h)

P_ort = matriz_ortogonal_xy()
P_persp = matriz_perspectiva(d=2.0)

print('Matriz ortogonal:\n', P_ort)
print('\nMatriz perspectiva (d=2.0):\n', P_persp)

ejemplo_h = a_homogeneas(puntos_3d[:, :3])
print('\nEjemplo de puntos en homogeneas (primeros 3):\n', ejemplo_h)


In [ ]:
def dibujar_2d(ax, proy, vertices_proy, titulo):
    ax.scatter(proy[0], proy[1], s=20, alpha=0.75)
    ax.scatter(vertices_proy[0], vertices_proy[1], s=50, color='crimson')
    for i, j in aristas:
        ax.plot([vertices_proy[0, i], vertices_proy[0, j]],
                [vertices_proy[1, i], vertices_proy[1, j]], color='crimson', linewidth=1.6)
    ax.set_title(titulo)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_aspect('equal', adjustable='box')

proy_ort = proyectar(puntos_3d, P_ort)
proy_persp = proyectar(puntos_3d, P_persp)

vert_ort = proyectar(vertices, P_ort)
vert_persp = proyectar(vertices, P_persp)

fig = plt.figure(figsize=(16, 5))
ax1 = fig.add_subplot(131, projection='3d')
ax2 = fig.add_subplot(132)
ax3 = fig.add_subplot(133)

ax1.scatter(puntos_3d[0], puntos_3d[1], puntos_3d[2], s=20, alpha=0.7)
for i, j in aristas:
    ax1.plot([vertices[0, i], vertices[0, j]],
             [vertices[1, i], vertices[1, j]],
             [vertices[2, i], vertices[2, j]], color='crimson', linewidth=1.6)
ax1.set_title('Original 3D')
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('Z')

dibujar_2d(ax2, proy_ort[:2], vert_ort[:2], 'Proyeccion ortogonal (XY)')
dibujar_2d(ax3, proy_persp[:2], vert_persp[:2], 'Proyeccion perspectiva (d=2.0)')

plt.tight_layout()
plt.show()


In [ ]:
distancias = [0.5, 1.0, 2.0, 4.0]
proyecciones = [proyectar(puntos_3d, matriz_perspectiva(d)) for d in distancias]
proyecciones_vertices = [proyectar(vertices, matriz_perspectiva(d)) for d in distancias]

# Limites comunes para comparar visualmente la escala entre valores de d
lim = max(np.max(np.abs(p[:2])) for p in proyecciones) * 1.1

fig, axes = plt.subplots(1, len(distancias), figsize=(18, 4.6))

for ax, d, proy, proy_v in zip(axes, distancias, proyecciones, proyecciones_vertices):
    dibujar_2d(ax, proy[:2], proy_v[:2], f'Perspectiva d={d}')
    ax.set_xlim(-lim, lim)
    ax.set_ylim(-lim, lim)

plt.tight_layout()
plt.show()

print('Observacion: en esta formulacion, al aumentar d, la proyeccion se ve mas grande (zoom in).')
